In [20]:
#First we import our data from the appropriate file
#Import the required libraries
import pandas as pd
import os.path as osp
import numpy as np

#Build the path for the data file
data_path = osp.join(
    osp.curdir,'VTS_2018_PUMF_CSV.xlsx')

#Use the read_excel function to pull data from the 'Sales Data' sheet
data = pd.read_excel(
    data_path,sheet_name='VTS_2018_PUMF')

# Drop Obs column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23758 entries, 0 to 23757
Columns: 269 entries, VPUMFID to VF_65P
dtypes: float64(1), int64(267), object(1)
memory usage: 48.8+ MB


In [21]:
data.head()

,VPUMFID,VQUARTER,VPRVENTP,VTPSZE,VRSN3P,VMODENTP,VRTEN,VRTEX,VCFARE1,VCFARE2,...,VM45_54P,VM55_64P,VM_65P,VF0_17P,VF18_24P,VF25_34P,VF35_44P,VF45_54P,VF55_64P,VF_65P
0,100000,3,35,5,1,2,1,1,6,6,...,0,0,0,1,0,1,0,0,0,0
1,100001,2,35,2,1,2,1,1,6,6,...,0,1,1,0,0,0,0,0,0,0
2,100002,4,35,1,10,1,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
3,100003,3,59,2,5,2,1,1,6,6,...,0,0,0,0,0,0,1,0,0,0
4,100004,2,24,1,2,1,2,2,2,2,...,1,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.model_selection import train_test_split
y=data['VTRNIN02']
X=data.drop(columns=['VTRNIN02','VPUMFID', 'VERDATE'])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [23]:
from sklearn.linear_model import LogisticRegression

logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(X_train, y_train)

LogisticRegression(C=1e+42, solver='liblinear')

In [24]:
#linear model

import statsmodels.api as sm
from statsmodels.formula.api import ols

X_train_1 = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train_1).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               VTRNIN02   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     3239.
Date:                Sun, 04 Jun 2023   Prob (F-statistic):               0.00
Time:                        16:36:32   Log-Likelihood:                -2331.2
No. Observations:               16630   AIC:                             5184.
Df Residuals:                   16369   BIC:                             7199.
Df Model:                         260                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6494      0.113     -5.723      0.0

In [25]:
#Ridge model first - make sure we optimize alpha!
import numpy as np
from sklearn import linear_model


reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 20))
reg.fit(X_train, y_train)
reg.alpha_

37.92690190732246

In [26]:
from sklearn import metrics
r2=metrics.r2_score(y_train, reg.predict(X_train))
print(r2)

-3625.513340221923


In [27]:
r2=metrics.r2_score(y_test, reg.predict(X_test))
print(r2)

-3692.879681358876


In [28]:
#let sklearn do it all for you 
reg2 = linear_model.LassoCV(cv=10)
reg2.fit(X_train, y_train)
reg2.alpha_

5003.970097133376

In [29]:
r2=metrics.r2_score(y_train, reg2.predict(X_train))
print(r2)

0.5596499206651775


In [30]:
r2=metrics.r2_score(y_test, reg2.predict(X_test))
print(r2)

0.5787360927370391


In [31]:
#elastic net has two parameters: you could gridsearch and let sklearn do its thing
reg3 = linear_model.ElasticNetCV(cv=10, l1_ratio = [.1, .5, .7, .9, .95, .99, 1])
reg3.fit(X_train, y_train)
reg3.alpha_

50039.70097133376

In [32]:
r2=metrics.r2_score(y_train, reg3.predict(X_train))
print(r2)

0.5596499206662792


In [33]:
r2=metrics.r2_score(y_test, reg3.predict(X_test))
print(r2)

0.578736092749587
